# <t><center>Assignment 4 - Exploratory Data Analysis</t>

### Data

COVID 19 Reported Patient Impact and Hospital Capacity - https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/anag-cw7u

This dataset provides facility level data for hospitalization aggregated on a weekly basis. Inlcudes information on weekly hospital capacity and useage (staffed general and ICU beds), inlcuding how much of that is being used by confirmed COVID-19 patients.

Data were downloaded using data portal tools to limit the file size. Data were filtered to only include facilities within California.

### EDA

In [83]:
import pandas as pd
import geopandas as gpd
import folium

In [84]:
df = pd.read_csv('COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility.csv')

c:\Users\chief\.conda\envs\geoprj\lib\site-packages\IPython\core\interactiveshell.py:3398: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [85]:
list(df.columns)

['hospital_pk',
 'collection_week',
 'state',
 'ccn',
 'hospital_name',
 'address',
 'city',
 'zip',
 'hospital_subtype',
 'fips_code',
 'is_metro_micro',
 'total_beds_7_day_avg',
 'all_adult_hospital_beds_7_day_avg',
 'all_adult_hospital_inpatient_beds_7_day_avg',
 'inpatient_beds_used_7_day_avg',
 'all_adult_hospital_inpatient_bed_occupied_7_day_avg',
 'inpatient_beds_used_covid_7_day_avg',
 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg',
 'total_adult_patients_hospitalized_confirmed_covid_7_day_avg',
 'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg',
 'total_pediatric_patients_hospitalized_confirmed_covid_7_day_avg',
 'inpatient_beds_7_day_avg',
 'total_icu_beds_7_day_avg',
 'total_staffed_adult_icu_beds_7_day_avg',
 'icu_beds_used_7_day_avg',
 'staffed_adult_icu_bed_occupancy_7_day_avg',
 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_avg',
 'staffed_icu_adult_patients_confirmed_covid_7_day_avg',
 'total_p

In [91]:
# this cell is to remove the columns that are not needed.
cols = ['hospital_pk',
        'collection_week',
        'state',
        'ccn',
        'hospital_name',
        'address',
        'city',
        'zip',
        'hospital_subtype',
        'fips_code',
        'is_metro_micro',
        'geocoded_hospital_address',
        'hhs_ids',
        ]

for col in list(df.columns):
    if (col not in cols) and ('7_day_avg' in col) and ('pediatric' not in col) and ('influenza' not in col):
                    cols.append(col)

df = df[cols]

hospital_pk
collection_week
state
ccn
hospital_name
address
city
zip
hospital_subtype
fips_code
is_metro_micro
total_beds_7_day_avg
all_adult_hospital_beds_7_day_avg
all_adult_hospital_inpatient_beds_7_day_avg
inpatient_beds_used_7_day_avg
all_adult_hospital_inpatient_bed_occupied_7_day_avg
inpatient_beds_used_covid_7_day_avg
total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg
total_adult_patients_hospitalized_confirmed_covid_7_day_avg
total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg
total_pediatric_patients_hospitalized_confirmed_covid_7_day_avg
inpatient_beds_7_day_avg
total_icu_beds_7_day_avg
total_staffed_adult_icu_beds_7_day_avg
icu_beds_used_7_day_avg
staffed_adult_icu_bed_occupancy_7_day_avg
staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_avg
staffed_icu_adult_patients_confirmed_covid_7_day_avg
total_patients_hospitalized_confirmed_influenza_7_day_avg
icu_patients_confirmed_influenza_7_day_avg
total_patients_hospi

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36012 entries, 0 to 36011
Data columns (total 28 columns):
 #   Column                                                                     Non-Null Count  Dtype  
---  ------                                                                     --------------  -----  
 0   hospital_pk                                                                36012 non-null  object 
 1   collection_week                                                            36012 non-null  object 
 2   state                                                                      36012 non-null  object 
 3   ccn                                                                        35910 non-null  object 
 4   hospital_name                                                              36012 non-null  object 
 5   address                                                                    36012 non-null  object 
 6   city                                                  

In [ ]:
# In the data documentation in notes that the data should be an aggregate of 7 days, reported every Friday. 
# Here I check to ensure that the data is being reported only on fridays.
df['collection_week'] = pd.to_datetime(df['collection_week'])
df['weekday'] = df['collection_week'].dt.day_name()
df['weekday'].unique()

In [113]:
print('Head of dataframe: \n' )
print(df.head(5))


Head of dataframe: 

  hospital_pk collection_week state      ccn                    hospital_name  \
0      050096      2022/05/06    CA  50096.0  WEST COVINA MEDICAL CENTER, INC   
1      052031      2022/05/06    CA  52031.0      BARLOW RESPIRATORY HOSPITAL   
2      052057      2020/11/06    CA  52057.0        SONOMA SPECIALTY HOSPITAL   
3      050194      2020/09/11    CA  50194.0   WATSONVILLE COMMUNITY HOSPITAL   
4      052057      2020/08/21    CA  52057.0        SONOMA SPECIALTY HOSPITAL   

               address         city    zip hospital_subtype  fips_code  ...  \
0     725 S ORANGE AVE  WEST COVINA  91790       Short Term       6037  ...   
1     2000 STADIUM WAY  LOS ANGELES  90026        Long Term       6037  ...   
2  501 PETALUMA AVENUE   SEBASTOPOL  95472        Long Term       6097  ...   
3    75 NIELSON STREET  WATSONVILLE  95076       Short Term       6087  ...   
4  501 PETALUMA AVENUE   SEBASTOPOL  95472        Long Term       6097  ...   

   inpatient_beds

In [114]:
print('Tail of dataframe: \n' )
print(df.tail(5))

Tail of dataframe: 

      hospital_pk collection_week state      ccn  \
36007      050069      2020/07/31    CA  50069.0   
36008      050785      2020/08/07    CA  50785.0   
36009      050224      2020/07/31    CA  50224.0   
36010      050684      2020/07/31    CA  50684.0   
36011      051301      2020/08/07    CA  51301.0   

                             hospital_name                 address  \
36007       PROVIDENCE ST. JOSEPH HOSPITAL    1100 WEST STEWART DR   
36008               DOCS SURGICAL HOSPITAL   6000 SAN VICENTE BLVD   
36009  HOAG MEMORIAL HOSPITAL PRESBYTERIAN          ONE HOAG DRIVE   
36010        MENIFEE GLOBAL MEDICAL CENTER  28400 MCCALL BOULEVARD   
36011    ADVENTIST HEALTH TEHACHAPI VALLEY           1100 MAGELLAN   

                city    zip           hospital_subtype  fips_code  ...  \
36007         ORANGE  92868                 Short Term       6059  ...   
36008    LOS ANGELES  90036                 Short Term       6037  ...   
36009  NEWPORT BEACH  9

In [112]:
print('A Random sample from the dataframe: \n' )
print(df.sample())

A Random sample from dataframe: 

      hospital_pk collection_week state      ccn  \
16435      051337      2021/08/27    CA  51337.0   

                         hospital_name           address       city    zip  \
16435  HAZEL HAWKINS MEMORIAL HOSPITAL  911 SUNSET DRIVE  HOLLISTER  95023   

                hospital_subtype  fips_code  ...  \
16435  Critical Access Hospitals       6069  ...   

       inpatient_beds_used_covid_7_day_avg  \
16435                                  7.7   

      total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg  \
16435                                                7.7                          

      total_adult_patients_hospitalized_confirmed_covid_7_day_avg  \
16435                                                7.7            

       inpatient_beds_7_day_avg  total_icu_beds_7_day_avg  \
16435                      25.0                       4.0   

       total_staffed_adult_icu_beds_7_day_avg  icu_beds_used_7_day_avg  \
164